#  Projet Kayak

#  A) Récupération de la localisation des villes de france ciblées (longitude et latitude) et de leur météo

In [1]:
import requests
import pandas as pd

villes_de_france = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg",
"Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence",
"Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz",
"Bayonne","La Rochelle"]

#Récupération de la localisation des villes 

class geolocalisation_cities():
    
    def __init__(self, cities_list):
        self.cities_list = cities_list
       
    
    def cities_france(self):
        
        dico ={}
        latitude =[]
        longitude =[]
        cities_liste =[]
        id_cities = []
        i=1
        
        for city in self.cities_list : 
            r=requests.get('https://nominatim.openstreetmap.org/search?q='+city+"&format=json&limit=1")
            geo = r.json()
        
            latitude.append(geo[0]["lat"])
            longitude.append(geo[0]["lon"])
            cities_liste.append(city)
            id_cities.append(i)
            i+=1
        
        dico["id_city"] = id_cities
        dico["city"] = cities_liste
        dico["latitude"] = latitude
        dico["longitude"] = longitude
    
        return dico

    
    def dataframe_cities(self):
        data = pd.DataFrame(self.cities_france())
        return data
    

# Récupération des données météorologiques sur 7 jours

class meteo_cities():
    
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    
    def meteo_dico(self):
        town= {}

        for index, row  in self.dataframe.iterrows():
            
            self.dataframe.loc[index,"city"] +" "+ self.dataframe.loc[index,"latitude"]
            city =  self.dataframe.loc[index,"city"]
            latitude1 =  self.dataframe.loc[index,"latitude"]
            longitude1 = self.dataframe.loc[index,"longitude"]

            rm= requests.get("https://api.openweathermap.org/data/2.5/onecall?lat="+latitude1+"&lon="+longitude1+"&units=metric&exclude=minutely,hourly&appid=2916ae0ed5269f82fa8b4a0e0553582a")
            data_json = rm.json()
            town[city] = data_json["daily"]
            
        return town
        
    
    
    def meteo_dataframe(self):
        meteo_final ={}
        cities =[]
        mean_cloud =[]
        mean_precipitation = []
        mean_temperature_day =[]
        id_city =[]
        index = 1



        
        precipitation =[]
        clouds =[]
        temperature_day =[]
        
        dico_meteo ={}
        

        for k, values in self.meteo_dico().items():
            for i, v in enumerate(self.meteo_dico()[k]):
                
                
                precipitation.append(v["pop"])
                clouds.append(v["clouds"])
                temperature_day.append(v["temp"]["day"])
    
           
            dico_meteo["Probability of precipitation"] = precipitation
            dico_meteo["clouds"] = clouds
            dico_meteo["day temperature"] = temperature_day

            data_meteo= pd.DataFrame(dico_meteo)
            mean_data = data_meteo.mean()
            cities.append(k)
            mean_cloud.append(mean_data.loc["clouds"])
            mean_precipitation.append(mean_data.loc["Probability of precipitation"])
            mean_temperature_day.append(mean_data.loc["day temperature"])
            id_city.append(index)
            index+=1
    
    
    
        meteo_final["id_city"]=id_city 
        meteo_final["city"] = cities
        meteo_final["Average probability of precipitation"] = mean_precipitation
        meteo_final["average cloud percentage"] = mean_cloud
        meteo_final["average daily temperatures"] = mean_temperature_day
        meteo_final_df = pd.DataFrame(meteo_final)
        
        return meteo_final_df
    
    

        
        
    

    
    
    

        
        
        
        
       

## 1) récupération de la localisation des villes de france

In [2]:
towns_l = geolocalisation_cities(villes_de_france)



In [3]:
# on récupère la latitude et la longitudes liée à chaque ville

geo_cities= towns_l.dataframe_cities()
geo_cities

,id_city,city,latitude,longitude
0,1,Mont Saint Michel,48.6359541,-1.511459954959514
1,2,St Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8588897,2.3200410217200766
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.584614,7.7507127
9,10,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195


## 2) Récupération des données métérologiques

In [4]:
# on récupère les données météorologiques liées à chaque ville

meteo = meteo_cities(geo_cities)
weather_cities = meteo.meteo_dataframe()

weather_cities

,id_city,city,Average probability of precipitation,average cloud percentage,average daily temperatures
0,1,Mont Saint Michel,0.536250,71.375000,16.407500
1,2,St Malo,0.526875,69.000000,16.246250
2,3,Bayeux,0.528750,69.416667,16.084583
3,4,Le Havre,0.513437,70.062500,15.947812
4,5,Rouen,0.503250,71.025000,16.068000
5,6,Paris,0.486042,72.000000,16.247708
6,7,Amiens,0.465536,73.839286,16.273214
7,8,Lille,0.447187,74.531250,16.257969
8,9,Strasbourg,0.446667,74.347222,16.373611
9,10,Chateau du Haut Koenigsbourg,0.448250,74.600000,16.225000


## 3) Fusion des deux dataframes localisation et météo des villes 

In [5]:
# on fusionne les deux dataframes l'une geo_cities (qui donne la localisation des villes) avec weather_cities qui donne (des informations sur la météo des villes)


data_weather_cities =geo_cities.merge(weather_cities, on = ['id_city',"city"])
data_weather_cities[['latitude', 'longitude']]=data_weather_cities[['latitude', 'longitude']].astype(float)
data_weather_cities

,id_city,city,latitude,longitude,Average probability of precipitation,average cloud percentage,average daily temperatures
0,1,Mont Saint Michel,48.635954,-1.511460,0.536250,71.375000,16.407500
1,2,St Malo,48.649518,-2.026041,0.526875,69.000000,16.246250
2,3,Bayeux,49.276462,-0.702474,0.528750,69.416667,16.084583
3,4,Le Havre,49.493898,0.107973,0.513437,70.062500,15.947812
4,5,Rouen,49.440459,1.093966,0.503250,71.025000,16.068000
5,6,Paris,48.858890,2.320041,0.486042,72.000000,16.247708
6,7,Amiens,49.894171,2.295695,0.465536,73.839286,16.273214
7,8,Lille,50.636565,3.063528,0.447187,74.531250,16.257969
8,9,Strasbourg,48.584614,7.750713,0.446667,74.347222,16.373611
9,10,Chateau du Haut Koenigsbourg,48.249490,7.344296,0.448250,74.600000,16.225000


## 4) Classement des villes en fonction de leur météo

In [6]:
# Classement des villes en fonction de la météo :

data_weather_cities_classified = data_weather_cities.sort_values(by=['Average probability of precipitation','average cloud percentage',"average daily temperatures"],ascending=[True,True,False])

data_weather_cities_classified

,id_city,city,latitude,longitude,Average probability of precipitation,average cloud percentage,average daily temperatures
34,35,La Rochelle,46.159113,-1.152043,0.356500,70.135714,19.226536
33,34,Bayonne,43.494514,-1.473666,0.362610,70.055147,19.266066
32,33,Biarritz,43.471144,-1.552727,0.363864,70.295455,19.227197
31,32,Montauban,44.017584,1.354999,0.364961,70.582031,19.209258
30,31,Toulouse,43.604462,1.444247,0.367379,70.830645,19.120806
29,30,Ariege,42.945537,1.406554,0.370958,70.945833,19.025125
28,29,Carcassonne,43.213036,2.349107,0.371810,71.241379,19.070043
27,28,Collioure,42.525050,3.083155,0.376696,71.312500,18.955313
26,27,Saintes Maries de la mer,43.452277,4.428717,0.382639,71.828704,18.831944
25,26,Aigues Mortes,43.565823,4.191284,0.389904,71.822115,18.755625


## 5) Présentation des top 5 destinations en France

In [7]:
top_5_destination = data_weather_cities_classified.iloc[0:5]
top_5_destination

,id_city,city,latitude,longitude,Average probability of precipitation,average cloud percentage,average daily temperatures
34,35,La Rochelle,46.159113,-1.152043,0.356500,70.135714,19.226536
33,34,Bayonne,43.494514,-1.473666,0.362610,70.055147,19.266066
32,33,Biarritz,43.471144,-1.552727,0.363864,70.295455,19.227197
31,32,Montauban,44.017584,1.354999,0.364961,70.582031,19.209258
30,31,Toulouse,43.604462,1.444247,0.367379,70.830645,19.120806


In [8]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio 
pio.renderers.default = "iframe_connected"
from plotly.subplots import make_subplots




access_token = 'pk.eyJ1IjoidmFuZXNzYTk3MjIzbSIsImEiOiJjbDBrMG5pYmEwMzI5M2RwZXhubXQxeG5lIn0._nVtqvdO2-Zq9QBkzUWHqw'
px.set_mapbox_access_token(access_token)



fig = px.scatter_mapbox(top_5_destination,title="The top 5 destinations in France for holidays the next 7 days !!!", lat="latitude", lon="longitude",     color="city", size="average cloud percentage")


fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    title_font_family ="Courier New",
    title_font_size = 25,
    title_font_color="blue",
    mapbox={"center": {"lon": 3, "lat" : 47}, "zoom": 4.8},
    
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
) 

fig.show()

# B) Scraping booking

## 1) Récupération des données des hôtels chez toutes les villes demandées

In [9]:
import requests
import pandas as pd
import json
import os


In [10]:
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

filename = "scrap_booking.json" 

if filename in os.listdir():
        os.remove(filename)

user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": { filename : {"format": "json"}
             }
})

In [11]:
import numpy as np
from urllib.parse import urlencode
from urllib.parse import urlparse
import re

class bookingSpider(scrapy.Spider):
    name = "booking"  
   
    cities = villes_de_france
    
    # va récupérer 3 pages par ville (page 0, 25, 60)
    
    start_urls = ["https://www.booking.com/searchresults.html"+ "?nflt=ht_id&" + urlencode({"ss":v,"offset":p}) for v in cities for p in [0,25,60]]
    allowed_domains = ['booking.com']
    
    try:
        def parse(self, response):
           
            print("procesing:"+response.url)
            
            # nom d'hôtels
            names = response.css('div.fcab3ed991.a23c043802::text').getall()
            
            # les notes attribuées aux hôtels
            notes = response.css('div.b5cd09854e.d10a6220b4::text').getall()
           
            # les liens urls
            all_links =  response.css('a.e13098a59f::attr(href)').getall()
           
                               
                              
            # nom des villes
            
            ville= response.url
            v = urlparse(ville)
            loc = re.sub(r"&offset=0|&offset=25|&offset=60", "", v.query[14:])
            loc = loc.replace("+"," ") 
            
                
            for name, note, link, in zip(names, notes, all_links) : 
                yield {'city':loc,
                    'hotel_name': name,
                       'score': note,
                      'url':link}
            
        
   
                
            
        
    except:
        pass
    
process.crawl(bookingSpider)
process.start()

hotels = pd.read_json('scrap_booking.json')

procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Le+Havre&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Le+Havre&offset=60
procesing:https://www.booking.com/searc

procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Biarritz&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Biarritz&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Biarritz&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayonne&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayonne&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayonne&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=La+Rochelle&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=La+Rochelle&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=La+Rochelle&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Toulouse&offset=0


In [12]:
hotels

,city,hotel_name,score,url
0,Bayeux,ibis budget Bayeux,8.2,https://www.booking.com/hotel/fr/etap-bayeux.e...
1,Bayeux,Hotel Reine Mathilde,8.4,https://www.booking.com/hotel/fr/reine-mathild...
2,Bayeux,Hôtel De Brunville et La Table du Grand Luxemb...,7.9,https://www.booking.com/hotel/fr/hoteldebrunvi...
3,Bayeux,Domaine de Bayeux,9.2,https://www.booking.com/hotel/fr/domaine-de-ba...
4,Bayeux,Château de Bellefontaine,8.1,https://www.booking.com/hotel/fr/chateau-de-be...
...,...,...,...,...
2497,Toulouse,Aparthotel Adagio Access Toulouse Jolimont,7.6,https://www.booking.com/hotel/fr/citea-toulous...
2498,Toulouse,Appartement Université-Capitole,8.4,https://www.booking.com/hotel/fr/appartement-c...
2499,Toulouse,Appartements avec chambre séparée - Toulouse h...,8.0,https://www.booking.com/hotel/fr/appartement-a...
2500,Toulouse,Le Clos des Salins,9.6,https://www.booking.com/hotel/fr/le-clos-des-s...


In [13]:
liste_url = hotels.url.tolist()
liste_url[0]

'https://www.booking.com/hotel/fr/etap-bayeux.en-gb.html?aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&nflt=ht_id&srpvid=e6ae458a558b0379&srepoch=1665309206&from_sustainable_property_sr=1&from=searchresults#hotelTmpl'

In [15]:
from bs4 import BeautifulSoup
import re



localisations =[]
descript=[]




for url1 in liste_url :
    head = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}

    reponse1 = requests.get(url1, headers=head)
    soup_bis= BeautifulSoup(reponse1.text,"html.parser")
    
  
        
    try:
        # récupération latitude et longitude
        localisation = soup_bis.find("a", id = "hotel_address")
        localisations.append([float(l) for l in localisation["data-atlas-latlng"].split(",")])
         
        
    except:
                    
        localisations.append(None)
        
        
    try:
        # description de l'hôtel
        description = soup_bis.find_all("div", class_ = "hp_desc_main_content")  
        d=[dc.get_text() for dc in description[0].find_all("p")]
        d =[re.sub(r"\n|\xa0&|\xa0|\xa0-", " ", w) for w in d]
        descript.append(d)
    except:
        
        descript.append(None)
        
        

In [16]:
len(localisations)

2502

In [101]:
# avec création des colonnes longitude et latitude et suppression de la colonne localisation

In [17]:
hotels["localisation"] = localisations

In [18]:
hotels["longitude"] =[l[1] for l  in hotels["localisation"]]
hotels["latitude"] =[l[0] for l in hotels["localisation"]]

In [19]:
hotels.drop("localisation",axis=1, inplace= True)

In [48]:
hotels["description"]=descript

In [55]:
def replace_none(x):
    if x == None:
        x = []
    return x

hotels["description"]= hotels["description"].apply(lambda x : replace_none(x))
    

In [56]:
dico_isnull = {c : (len(hotels[hotels[c].isnull()== True])/(hotels.shape[0])) * 100 for c in hotels.columns}
dico_isnull

{'city': 0.0,
 'hotel_name': 0.0,
 'score': 0.0,
 'url': 0.0,
 'longitude': 0.0,
 'latitude': 0.0,
 'description': 0.0}

In [57]:

hotels["description"] = hotels["description"].apply(lambda x: ' '.join(map(str, x)))
    


In [58]:
hotels

,city,hotel_name,score,url,longitude,latitude,description
0,Bayeux,ibis budget Bayeux,8.2,https://www.booking.com/hotel/fr/etap-bayeux.e...,-0.646487,49.254242,"Between Caen and Cherbourg, near the exit 36 o..."
1,Bayeux,Hotel Reine Mathilde,8.4,https://www.booking.com/hotel/fr/reine-mathild...,-0.701725,49.276032,Hotel Reine Mathilde is situated in the mediev...
2,Bayeux,Hôtel De Brunville et La Table du Grand Luxemb...,7.9,https://www.booking.com/hotel/fr/hoteldebrunvi...,-0.703514,49.278158,You're eligible for a Genius discount at Hôtel...
3,Bayeux,Domaine de Bayeux,9.2,https://www.booking.com/hotel/fr/domaine-de-ba...,-0.698510,49.272326,"Located in the centre of Bayeux, the Domaine d..."
4,Bayeux,Château de Bellefontaine,8.1,https://www.booking.com/hotel/fr/chateau-de-be...,-0.688840,49.269740,You're eligible for a Genius discount at Châte...
...,...,...,...,...,...,...,...
2497,Toulouse,Aparthotel Adagio Access Toulouse Jolimont,7.6,https://www.booking.com/hotel/fr/citea-toulous...,1.463062,43.614759,This Adagio Access Hotel is located in Toulous...
2498,Toulouse,Appartement Université-Capitole,8.4,https://www.booking.com/hotel/fr/appartement-c...,1.437645,43.609803,Appartement Université-Capitole in Toulouse pr...
2499,Toulouse,Appartements avec chambre séparée - Toulouse h...,8.0,https://www.booking.com/hotel/fr/appartement-a...,1.444821,43.605947,You're eligible for a Genius discount at Appar...
2500,Toulouse,Le Clos des Salins,9.6,https://www.booking.com/hotel/fr/le-clos-des-s...,1.446073,43.594414,"In the heart of Toulouse historic city centre,..."


## 2) Présentation finale des données classées des hôtels de chaque ville en dataframe pandas

In [63]:


hotels = hotels.sort_values(by = ['city'])
t = [hotels[hotels.city == v].sort_values(by = ['score'],ascending = False)for v in villes_de_france]
t[5]

,city,hotel_name,score,url,longitude,latitude,description
451,Paris,Millésime Hôtel,9.2,https://www.booking.com/hotel/fr/millesime-par...,2.335039,48.855111,Located in a 17th century building in the hear...
399,Paris,Résidence Charles Floquet,9.1,https://www.booking.com/hotel/fr/residence-cha...,2.294172,48.856100,You're eligible for a Genius discount at Résid...
370,Paris,Résidence Charles Floquet,9.1,https://www.booking.com/hotel/fr/residence-cha...,2.294172,48.856100,You're eligible for a Genius discount at Résid...
388,Paris,Apartments Du Louvre - Le Marais,9.0,https://www.booking.com/hotel/fr/apartments-du...,2.350519,48.863615,You're eligible for a Genius discount at Apart...
440,Paris,Hotel Taylor,8.9,https://www.booking.com/hotel/fr/taylor.en-gb....,2.359106,48.869397,You're eligible for a Genius discount at Hotel...
...,...,...,...,...,...,...,...
392,Paris,Hôtel Mondial,7.3,https://www.booking.com/hotel/fr/mondial-paris...,2.343943,48.871888,You're eligible for a Genius discount at Hôtel...
375,Paris,Tonic Hotel Saint Germain des Prés,7.3,https://www.booking.com/hotel/fr/tonic-saint-g...,2.337393,48.851698,Tonic Hotel Saint Germain des Prés is located ...
393,Paris,Les Patios du Marais 1,7.3,https://www.booking.com/hotel/fr/les-patios-du...,2.356815,48.864264,You're eligible for a Genius discount at Les P...
378,Paris,Aparthotel Adagio Paris XV,6.8,https://www.booking.com/hotel/fr/parisxv.en-gb...,2.282343,48.831309,Aparthotel Adagio Paris XV is located in Paris...


In [64]:
#fusion de toutes les dataframes d'hôtels de chaque ville en un seul dataframe finale

info_hotels_by_city  = pd.concat(t)

info_hotels_by_city 

,city,hotel_name,score,url,longitude,latitude,description
75,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in..."
163,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f..."
161,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa..."
154,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...
94,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l..."
...,...,...,...,...,...,...,...
2474,La Rochelle,greet Hotel La Rochelle Centre,7.2,https://www.booking.com/hotel/fr/greet-la-roch...,-1.141466,46.156780,"Featuring a seasonal outdoor swimming pool, a ..."
2455,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10..."
2421,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10..."
2456,La Rochelle,Hôtel Henri IV,6.8,https://www.booking.com/hotel/fr/henri-iv.en-g...,-1.151601,46.159023,Set in the heart of the city centre and histor...


In [69]:
# Ajout de l'id_city dans la dataframe

def id_city (x): 
    if x == "Mont Saint Michel":
        x = 1
    if x == "St Malo":
        x = 2
    if x == "Bayeux":
        x = 3
    if x == "Le Havre":
        x=4
    if x== "Rouen":
        x=5
    if x== "Paris":
        x=6
    if x== "Amiens":
        x=7
    if x=="Lille":
        x=8
    if x =="Strasbourg":
        x=9
    if x=="Chateau du Haut Koenigsbourg":
        x=10
    if x== "Colmar":
        x=11
    if x=="Eguisheim":
        x=12
    if x=="Besancon":
        x=13
    if x=="Dijon":
        x=14
    if x=="Annecy":
        x=15
    if x=="Grenoble":
        x=16
    if x=="Lyon":
        x=17
    if x=="Gorges du Verdon":
        x=18
    if x=="Bormes les Mimosas":
        x=19
    if x=="Cassis":
        x=20
    if x=="Marseille":
        x=21
    if x=="Aix en Provence":
        x=22
    if x=="Avignon":
        x=23
    if x=="Uzes":
        x=24
    if x=="Nimes":
        x=25
    if x=="Aigues Mortes":
        x=26
    if x== "Saintes Maries de la mer":
        x=27
    if x== "Collioure":
        x=28
    if x=="Carcassonne":
        x=29
    if x=="Ariege":
        x=30
    if x=="Toulouse":
        x=31
    if x=="Montauban":
        x=32
    if x=="Biarritz":
        x=33
    if x=="Bayonne":
        x=34
    if x=="La Rochelle":
        x=35
    
    return x

In [72]:
info_hotels_by_city ["id_city"] = info_hotels_by_city ["city"].apply(lambda x : id_city(x))

In [93]:
info_hotels_by_city # tableau final sur les hôtels par ville

,city,hotel_name,score,url,longitude,latitude,description,id_city
75,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1
163,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f...",1
161,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa...",1
154,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...,1
94,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l...",1
...,...,...,...,...,...,...,...,...
2474,La Rochelle,greet Hotel La Rochelle Centre,7.2,https://www.booking.com/hotel/fr/greet-la-roch...,-1.141466,46.156780,"Featuring a seasonal outdoor swimming pool, a ...",35
2455,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35
2421,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35
2456,La Rochelle,Hôtel Henri IV,6.8,https://www.booking.com/hotel/fr/henri-iv.en-g...,-1.151601,46.159023,Set in the heart of the city centre and histor...,35


## 3) Fusion des deux tableaux "info_hotels_by_city" (info sur les hôtels par ville) "data_weather_cities_classified"(info météorologique par ville)

In [79]:
hotels_cities_weather =info_hotels_by_city.merge(data_weather_cities_classified, on=["id_city","city"], how = "outer")
hotels_cities_weather

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
1,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
2,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
3,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...,1,48.635954,-1.511460,0.53625,71.375000,16.407500
4,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,La Rochelle,greet Hotel La Rochelle Centre,7.2,https://www.booking.com/hotel/fr/greet-la-roch...,-1.141466,46.156780,"Featuring a seasonal outdoor swimming pool, a ...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2498,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2499,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2500,La Rochelle,Hôtel Henri IV,6.8,https://www.booking.com/hotel/fr/henri-iv.en-g...,-1.151601,46.159023,Set in the heart of the city centre and histor...,35,46.159113,-1.152043,0.35650,70.135714,19.226536


## 4) Récupération des 20 meilleurs hôtels de la région des 35 villes 

In [94]:
# Récupération des 20 meilleurs hôtels de la région des 35 villes 

liste_hotels_all= [hotels_cities_weather[hotels_cities_weather.city == town].iloc[0:20] for town in villes_de_france]


best_hotels_all = pd.concat(liste_hotels_all)
best_hotels_all 

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
1,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
2,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
3,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...,1,48.635954,-1.511460,0.53625,71.375000,16.407500
4,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,La Rochelle,LA ROCHELAISE : Appartement calme & somptueux ...,8.9,https://www.booking.com/hotel/fr/la-rochelaise...,-1.150428,46.159773,"Offering free WiFi and city views, LA ROCHELAI...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2447,La Rochelle,Les Apparts de Charly-Le 97,8.8,https://www.booking.com/hotel/fr/le-97.en-gb.h...,-1.139312,46.167725,You're eligible for a Genius discount at Les A...,35,46.159113,-1.152043,0.35650,70.135714,19.226536
2448,La Rochelle,Hôtel Le Transatlantique - Lycée Hôtelier La R...,8.7,https://www.booking.com/hotel/fr/lycee-hotelie...,-1.160263,46.143734,"Located in La Rochelle, 2.5 km from Parc Expo ...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2449,La Rochelle,Central Park Hôtel & Spa,8.7,https://www.booking.com/hotel/fr/central-park-...,-1.160823,46.158913,You're eligible for a Genius discount at Centr...,35,46.159113,-1.152043,0.35650,70.135714,19.226536


In [100]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio 
pio.renderers.default = "iframe_connected"
from plotly.subplots import make_subplots

access_token = 'pk.eyJ1IjoidmFuZXNzYTk3MjIzbSIsImEiOiJjbDBrMG5pYmEwMzI5M2RwZXhubXQxeG5lIn0._nVtqvdO2-Zq9QBkzUWHqw'

import plotly.express as px


fig = px.scatter_mapbox(best_hotels_all,title="The Top-20 hotels for the 35 cities!!!", lat="latitude_x", lon="longitude_x", color="average daily temperatures", size = "score", mapbox_style="carto-positron")


fig.update_layout(
    title_x=0.5,
    title_y=1,
    title_font_family ="Courier New",
    title_font_size = 25,
    title_font_color="purple",
    mapbox={"center": {"lon": 3, "lat" : 47}, "zoom": 4.8},

    margin={"l": 0, "r": 0, "b": 0, "t": 50}
) 

fig.show()

# C) Stockage du tableau de données météorologiques et des hotels de chaque ville en seul fichier csv

In [96]:
# On créer un fichier csv de notre tableau final qui donne des infos sur les villes, les hôtels et la météo

hotels_cities_weather.to_csv('hotels_cities_weather.csv',index = False) 




In [97]:
hcw = pd.read_csv("hotels_cities_weather.csv")
hcw

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
1,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
2,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
3,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...,1,48.635954,-1.511460,0.53625,71.375000,16.407500
4,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,La Rochelle,greet Hotel La Rochelle Centre,7.2,https://www.booking.com/hotel/fr/greet-la-roch...,-1.141466,46.156780,"Featuring a seasonal outdoor swimming pool, a ...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2498,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2499,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2500,La Rochelle,Hôtel Henri IV,6.8,https://www.booking.com/hotel/fr/henri-iv.en-g...,-1.151601,46.159023,Set in the heart of the city centre and histor...,35,46.159113,-1.152043,0.35650,70.135714,19.226536


# D) Envoie du fichier csv dans le bucket S3 

In [102]:
pip install boto3

  Using cached boto3-1.21.42-py3-none-any.whl (132 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Using cached botocore-1.24.42-py3-none-any.whl (8.7 MB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import boto3

# Création d'un bucket sur AWS

session = boto3.Session(aws_access_key_id="", aws_secret_access_key="")
s3 = session.resource("s3")
s3.create_bucket(Bucket="bucketkayak972234")






In [ ]:


## Ajouter un fichier csv dans le S3

data = open('hotels_cities_weather.csv', 'rb')
s3.Bucket('bucketkayak972234').put_object(Key='hotels_cities_weather.csv', Body=data) # stock le fichier csv dans le bucket "bucketkayak972234" dans le service S3


In [107]:
# télécharger le fichier csv dans le S3

bucket1 = s3.Bucket('bucketkayak972234')
bucket1.download_file('hotels_cities_weather.csv', 'hotels_cities_weather.csv')

In [99]:
weather_hotels_cities = pd.read_csv("hotels_cities_weather.csv")
weather_hotels_cities

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
1,Mont Saint Michel,le coin des hirondelles,9.6,https://www.booking.com/hotel/fr/le-coin-des-h...,-1.474069,48.602864,"Featuring free bikes, a garden, and barbecue f...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
2,Mont Saint Michel,Le Canari,9.6,https://www.booking.com/hotel/fr/le-canari.en-...,-1.456738,48.610084,"Le Canari, a property with a terrace, is situa...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
3,Mont Saint Michel,Le Beauvoir,9.6,https://www.booking.com/hotel/fr/le-beauvoir-b...,-1.513098,48.597713,You're eligible for a Genius discount at Le Be...,1,48.635954,-1.511460,0.53625,71.375000,16.407500
4,Mont Saint Michel,Villa Jacuzzi au pied de la Baie du Mont St Mi...,9.5,https://www.booking.com/hotel/fr/villa-jacuzzi...,-1.469224,48.683844,"Boasting a hot tub, Villa Jacuzzi au pied de l...",1,48.635954,-1.511460,0.53625,71.375000,16.407500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,La Rochelle,greet Hotel La Rochelle Centre,7.2,https://www.booking.com/hotel/fr/greet-la-roch...,-1.141466,46.156780,"Featuring a seasonal outdoor swimming pool, a ...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2498,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2499,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.35650,70.135714,19.226536
2500,La Rochelle,Hôtel Henri IV,6.8,https://www.booking.com/hotel/fr/henri-iv.en-g...,-1.151601,46.159023,Set in the heart of the city centre and histor...,35,46.159113,-1.152043,0.35650,70.135714,19.226536


# E) Stocker le fichier csv dans la base de donnée SQL par AWS (RDS)

In [108]:
!pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)


In [109]:
from sqlalchemy import create_engine  

hostname = '' 
password = '' 
username = '' 
dbname = ''  # Replace YOUR_USERNAME, YOUR_PASSWORD and YOUR_HOSTNAME # by your credentials 

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{hostname}/{dbname}", echo=True)

In [112]:
# envoie du fichier csv  pandas dans la base de donnée SQL

weather_hotels_cities.to_sql('liste_hotels_par_ville', con=engine)

2022-04-18 14:39:52,768 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-18 14:39:52,770 INFO sqlalchemy.engine.Engine [cached since 178.5s ago] {'name': 'liste_hotels_par_ville'}
2022-04-18 14:39:52,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-18 14:39:52,795 INFO sqlalchemy.engine.Engine 
CREATE TABLE liste_hotels_par_ville (
	index BIGINT, 
	id_city BIGINT, 
	city TEXT, 
	latitude_x FLOAT(53), 
	longitude_x FLOAT(53), 
	"Average probability of precipitation" FLOAT(53), 
	"average cloud percentage" FLOAT(53), 
	"average daily temperatures" FLOAT(53), 
	"nom_de_hôtel" TEXT, 
	note FLOAT(53), 
	descriptif TEXT, 
	points_forts TEXT, 
	url TEXT, 
	longitude_y FLOAT(53), 
	latitude_y FLOAT(53)
)


2022-04-18 14:39:52,796 INFO sqlalchemy.engine.Engine [no key 0.00135s] {}
2022-04-18 14:39:52,815 INFO sqlalchemy.engine.Engine CREATE INDEX ix_liste_ho

131